In [ ]:
%matplotlib inline

import pandas as pd
import numpy as np
from scipy.stats import rv_continuous, norm, truncnorm, arcsine, chi2, gamma, triang, entropy
import matplotlib.pyplot as plt
from itertools import combinations_with_replacement
from time import time

In [ ]:
gift_types = ['horse', 'ball', 'bike', 'train', 'coal', 'book', 'doll', 'blocks', 'gloves']
gift_numbers = [1000, 1100, 500, 1000, 166, 1200, 1000, 1000, 200]
gift_ids = [[item +'_'+str(i) for i in range(gift_numbers[index])] for index, item in enumerate(gift_types)]
df = pd.read_csv('best_combinations.csv')

In [ ]:
def unshared_copy(inList):
    if isinstance(inList, list):
        return list( map(unshared_copy, inList) )
    return inList

def max_number_of_packages(recipe, numbers):
    max_packs = np.inf
    for rn,gn in zip(recipe, numbers):
        if rn == 0:
            pass
        elif gn/rn < max_packs:
            max_packs = gn/rn    
    return int(max_packs)

def pack_recipe_to_str(package_recipe, gift_types, ids):
    out = ''
    for pack_n, gt, gift_id_list in zip(package_recipe, gift_types, ids):
        for i in range(int(pack_n)):
            out += gift_id_list.pop() + ' '
    return out[:-1]


def greedy_gnome(dataframe, gift_types, gift_numbers, gift_ids):
    internal_ids = unshared_copy(gift_ids)
    internal_numbers = unshared_copy(gift_numbers)
    packages = []
    scores = []
    package_counter = max_packages = 1000
    df = dataframe.sort_values('score', ascending = False)
    score = 0
    for row in df.iterrows():
        package_recipe = row[1][gift_types]
        m = min(max_number_of_packages(package_recipe, internal_numbers), package_counter)
        internal_numbers -= package_recipe*m
        package_counter -= m
        for i in range(m):
            packages += [pack_recipe_to_str(package_recipe, gift_types, internal_ids)]
        scores += [row[1]['score']] * m
        if package_counter <= 0:
            break
    return [scores, packages, internal_numbers]

def pack_recipe_to_str_random(package_recipe, gift_types, ids):
    out = ''
    for pack_n, gift_id_list in zip(package_recipe, ids):
        for i in range(int(pack_n)):
            random_index = np.random.randint(len(gift_id_list))
            out += gift_id_list[random_index] + ' '
            del gift_id_list[random_index]
    return out[:-1]


def greedy_gnome_random(dataframe, gift_types, gift_numbers, gift_ids):
    internal_ids = unshared_copy(gift_ids)
    internal_numbers = unshared_copy(gift_numbers)
    packages = []
    scores = []
    package_counter = max_packages = 1000
    df = dataframe.sort_values('score', ascending = False)
    score = 0
    for row in df.iterrows():
        package_recipe = row[1][gift_types]
        m = min(max_number_of_packages(package_recipe, internal_numbers), package_counter)
        internal_numbers -= package_recipe*m
        package_counter -= m
        for i in range(m):
            packages += [pack_recipe_to_str_random(package_recipe, gift_types, internal_ids)]
        scores += [row[1]['score']] * m
        if package_counter <= 0:
            break
    return [scores, packages, internal_numbers]

def fast_greedy_gnome(dataframe, gift_types, gift_numbers, N_packs=1000):
    j = 0
    cart_recipe = [None] * N_packs
    package_counter = 0
    items_left = gift_numbers[:]
    scores = []
    for i in dataframe.itertuples():
        index = i[0]
        recipe = i[1:10]
        score = i[10]
        n_items = i[11]
        max_packs = max_number_of_packages(recipe, items_left)
        number_of_packages = min(max_packs, max(0,N_packs-package_counter))
        if number_of_packages == 0:
            continue
        #print('max_packs = %d, number_of_packages = %d, package_counter = %d' 
        #      % (max_packs, number_of_packages, package_counter))
        for index, [r, il] in enumerate(zip(recipe, items_left)):
            items_left[index] = il - r * number_of_packages
        scores += [score] * number_of_packages
        cart_recipe[package_counter:package_counter+number_of_packages] = [recipe] * number_of_packages
        package_counter += number_of_packages
        #print(recipe)
        #print(items_left)
        #print('----------')
        if package_counter == 1000:
            break
    return [scores, cart_recipe, items_left]

In [ ]:
max_i = 7
gg1 = greedy_gnome(df[(df.n_items <= max_i) & (df.n_items >= 3)], gift_types, gift_numbers, gift_ids)

In [ ]:
sum(gg1[0])

In [ ]:
def gift_list_csv(name, pack_lists):
    pd.DataFrame(pack_lists, columns = ['Gifts']).to_csv(name, index=False)

In [ ]:
gift_list_csv('first_normal.csv', gg1[1])

In [ ]:
first_normal_score = 35158.63183

In [ ]:
max_i = 7
random_gg_1 = greedy_gnome_random(df[(df.n_items <= max_i) & (df.n_items >= 3)], gift_types, gift_numbers, gift_ids)

In [ ]:
#gift_list_csv('first_ggr.csv', random_gg_1[1])
#gift_list_csv('second_ggr.csv', random_gg_2[1])

In [ ]:
first_ggr_score = 35142.35864
second_ggr_score = 35077.81176

In [ ]:
max_sum = 0
max_ij = -1
for i in [10*t for t in range(100)]:
    print('========\ni = %d' % i)
    for j in [10*t for t in range(i,100)]:
        gg = fast_greedy_gnome(df[(df.max_packs >= 60) & (df.n_items >= 3) & (df.ball>=i) & (df.ball<=j)& (df.gloves>=i) & (df.gloves<=j)], gift_types, gift_numbers)
        sum_i = sum(gg[0])
        #print('i = %d' % i)
        #print('Expected score = %.2f\nMax score = %.2f\n----------' % (sum_i, max_sum))
        if sum_i > max_sum:
            max_sum = sum_i
            max_ij = [i, j]
            
print(max_ij)
print(max_sum)

In [ ]:
max_i = -1
max_sum = 0
for i in range(3,28):
    gg = fast_greedy_gnome(df[(df.max_packs >= 60) & (df.n_items >= 3) & (df.n_items<=7) & (df.score>i)], gift_types, gift_numbers)
    sum_i = sum(gg[0])
    #print('i = %d' % i)
    #print('Expected score = %.2f\nMax score = %.2f\n----------' % (sum_i, max_sum))
    if sum_i > max_sum:
        max_sum = sum_i
        max_i = i
        
print(max_i)
print(max_sum)

In [ ]:
max_i

In [ ]:
df[df.max_packs<60]

In [ ]:
df_a = df

In [ ]:
df_a['scores'] = 0

In [ ]:
df_a['score'] = 0

In [ ]:
def choiceful_gnome(dataframe, gift_types, gift_numbers, gift_types, N_packs=1000):
    n = 0
    items_left = gift_numbers[:]
    while n < N_packs:

In [ ]:
df['score'][1]

In [ ]:
a=df.apply(lambda x: x[9]*min([(n_left//n_recipe)if n_recipe>0 else np.inf  for n_left, n_recipe in zip(gift_numbers, x[:9])]),axis=1)

In [ ]:
def foo(a,b):
    if b == 0:
        return np.inf
    else:
        return a//b
    
def bar(recipe, gift_numbers):
    return min([foo(n_left,n_recipe) for n_left, n_recipe in zip(gift_numbers, recipe)])
    
df.apply(lambda x:bar(x, gift_numbers), axis=1)

In [ ]:
a = df.apply(lambda x: x[9]*min([(n_left//n_recipe)if n_recipe>0 else np.inf  for n_left, n_recipe in zip(gift_numbers, x[:9])]),axis=1)
a.rename('total_score')
b = df['max_packs']
c = pd.concat([a,b],axis = 1)

In [ ]:
c.columns = ['total_score', 'max_packs']
c.sort_values('total_score',ascending=False)

In [ ]:
a = df[gift_types].loc[0][0:9]

In [ ]:
int(np.inf)